# Week 3: Fitting

In many different cases, we might have a model for how a system works, and want to fit that model to a set of observations. 

We're going to investigate the process of fitting using a classic paper that proposed a model for the [T cell receptor](https://www.ncbi.nlm.nih.gov/pubmed/11606269). Here, the authors develop a mathematical model for how binding occurs and then have observations of how much binding occurs under specific conditions. Identifying whether and how this model fits has led to a better understanding of how our immune system recognizes diseased cells, and how to design T cells that respond to diseases like cancer.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import least_squares

np.seterr(over='raise')


def Req_func(Phisum, Rtot, L0, KxStar, f, Ka):
    """ Mass balance. Transformation to account for bounds. """
    Req = Rtot / (1.0 + L0 * f * Ka * (1 + Phisum) ** (f - 1))
    return Phisum - Ka * KxStar * Req


def StoneMod(Rtot: float, Kd: float, v, Kx: float, L0: np.ndarray):
    '''
    Returns the number of mutlivalent ligand bound to a cell with Rtot
    receptors, granted each epitope of the ligand binds to the receptor
    kind in question with dissociation constant Kd and cross-links with
    other receptors with crosslinking constant Kx. All eq derived from Stone et al. (2001).
    '''
    v = np.int_(v)
    assert L0.shape == v.shape

    if Rtot <= 0.0:
        raise RuntimeError("You input a negative amount of receptor.")
    
    if Kx <= 0.0:
        raise RuntimeError("You input a negative Kx.")

    if np.amin(L0) <= 0.0:
        raise RuntimeError("You input a negative L0.")

    Ka = 1.0 / Kd
    KxStar = Kx / Ka

    ## Solve Req by calling least_squares
    lsq = least_squares(Req_func, np.ones_like(L0), jac="cs",
                        max_nfev=5000, xtol=1.0E-10, ftol=1.0E-10, gtol=1.0E-10,
                        args=(Rtot, L0, KxStar, v, Ka))

    if lsq['success'] is False:
        print(lsq)
        raise RuntimeError("Failure in solving for Req. If you see this message contact Dr. Meyer.")

    Phisum = lsq.x

    # Calculate L, according to equation 7
    Lbound = L0 / KxStar * ((1 + Phisum) ** v - 1)

    # Calculate Rmulti from equation 5
    Rmulti = L0 / KxStar * v * Phisum * ((1 + Phisum) ** (v - 1) - 1)

    # Calculate Rbound
    Rbnd = L0 / KxStar * v * Phisum * (1 + Phisum) ** (v - 1)

    return Lbound, Rbnd, Rmulti

Xs = np.array([8.1E-11, 3.4E-10, 1.3E-09, 5.7E-09, 2.1E-08, 8.7E-08, 3.4E-07, 1.5E-06, 5.7E-06, 2.82E-11, 1.17E-10, 4.68E-10, 1.79E-09, 7.16E-09, 2.87E-08, 1.21E-07, 4.5E-07, 1.87E-06, 1.64E-11, 6.93E-11, 2.58E-10, 1.11E-09, 4.35E-09, 1.79E-08, 7.38E-08, 2.9E-07, 1.14E-06])
Ys = np.array([-196, -436, 761, 685, 3279, 7802, 11669, 12538, 9012, -1104, -769, 1455, 2693, 7134, 11288, 14498, 16188, 13237, 988, 1734, 4491, 9015, 13580, 17159, 18438, 18485, 17958])
Vs = np.repeat([2, 3, 4], 9)

#### (1) We will fit the data contained within Fig. 3B. Plot this data and describe the relationship you see between Kx, Kd, and valency.

In [ ]:
# Note that Xs are the concentrations from the experiment
# Vs are the complex valencies
# Ys are the measurements of response
# Each of these vectors is the same length, with each position describing an experimental measurement

def plot_valency_data(Xs_in, Ys_in, Vs_in, format='o-'):
    """If you pass in your real or simulated data, this will plot it for you."""
    colors = ['r', 'g', 'b']

    for valency in range(3):
        plt.semilogx(Xs_in[Vs_in == valency + 2], 
                     Ys_in[Vs_in == valency + 2],
                     colors[valency] + format)
            
    plt.xlim([1E-11, 1E-5])
    plt.xticks([1E-10, 1E-8, 1E-6])
    plt.xlabel('[Oligomer] M')

# Answer

#### (2) First, to do so, we'll need a function that takes the model predictions, scales them to the units of the actual measurements, and finds the predictions for each condition. Define a scaling parameter and a function that takes it along with the other parameters to make predictions about the experiment.

Use the fit parameters shown in Table 1 (row 2) and overlay with the measurements to ensure your function is working. (Scale = 1 for now.)

In [ ]:
# Here is a scaling function
# Note that the unknown parameters are passed in as an array
# This will make working with least_squares() easier later
# parameters = [scale, Kd (M), Kx (cell)]
def scale_StoneMod(parameters, Vs, Xs):
    Rtot = 24000.0 # cell^-1
    Lbound, Rbnd, Rmulti = StoneMod(Rtot, parameters[1], Vs, parameters[2], Xs)
    return parameters[0] * Rmulti

# Answer

#### (3) Now use `scipy.optimize.least_squares` to find the least squares solution.

In [ ]:
# You will need a function that returns the residuals:
def residuals(parameters, Vs_in, Xs_in, Ys_in):
    return Ys_in - scale_StoneMod(parameters, Vs_in, Xs_in)

# Note that the data (Xs, Ys) are not unknowns, so they should be passed in using the args argument of least_squares

# With this function, your least squares call should look something like this:
# lsq_solution = least_squares(residuals, params_guess, args=(Vs, Xs, Ys))

# Answer

#### (4) Using leave-one-out crossvalidation, does this model predict the data? Plot the measured vs. predicted data.

In [ ]:
# scikit-learn provides a great interface for getting indices that help you split the data
from sklearn.model_selection import LeaveOneOut

for train_idx, test_idx in LeaveOneOut().split(Vs):
    Vs_training = Vs[train_idx]
    Vs_testing = Vs[test_idx]

    # Xs_training...

# Answer

#### (5) Using bootstrap estimation, plot the confidence interval of the model predictions along with the data points.
"Confidence interval" does not have a precise definition. For example, you could show the interval over which 50% of the bootstrap samples fall (25th to 75th quantile).

In [ ]:
# sklearn has a very useful resample function
from sklearn.utils import resample

# For example, to resample the data, you can do this:
resamp = resample(range(Vs.size))
Vs_resampled = Vs[resamp]
Xs_resampled = Xs[resamp]
Ys_resampled = Ys[resamp]

# Answer

#### (6) _Generally_, how would you expect the cross-validation and bootstrap results to change if you had fewer data points?

Explain your answer.

Answer.

#### (7) Now, we will perform a local sensitivity analysis to look at the dependence of the model results on each parameter. Vary each parameter up and down relative to its optimum by 10-fold **while holding the others constant**, and plot the sum of squared error. Which parameter affects the error the most? Which one the least?

You should get a plot that roughly looks like a "U" for each parameter. Use a log-log plot to show the variation the best (`plt.loglog`).

In [ ]:
# Answer